# 🚀 Deploy and Customize `amazon/chronos-2` Foundation Model on Time-Series Forecasting

<div style="background-color: #f0f8ff; border-left: 4px solid #0073bb; padding: 10px; margin-bottom: 15px;">
<strong>⚙️ Recommended instance type:</strong> <code>ml.c5.2xlarge</code>
</div>


**Chronos-2** is a foundation model for time series forecasting that builds on [Chronos](https://arxiv.org/abs/2403.07815) and [Chronos-Bolt](https://aws.amazon.com/blogs/machine-learning/fast-and-accurate-zero-shot-forecasting-with-chronos-bolt-and-autogluon/). It offers significant improvements in capabilities and can handle diverse forecasting scenarios not supported by earlier models.

| Capability | Chronos | Chronos-Bolt | Chronos-2 |
|------------|---------|--------------|-----------|
| Univariate Forecasting | ✅ | ✅ | ✅ |
| Cross-learning across items | ❌ | ❌ | ✅ |
| Multivariate Forecasting | ❌ | ❌ | ✅ |
| Past-only (real/categorical) covariates | ❌ | ❌ | ✅ |
| Known future (real/categorical) covariates | 🧩 | 🧩 | ✅ |
| Fine-tuning support | ✅ | ✅ | ✅ |
| Max. Context Length | 512 | 2048 | 8192 |

🧩 Chronos/Chronos-Bolt do not natively support future covariates, but they can be combined with external covariate regressors (see [AutoGluon tutorial](https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-chronos.html#incorporating-the-covariates)). This only models per-timestep effects, not effects across time. In contrast, Chronos-2 supports all covariate types natively.

More details about Chronos-2 are available in the [technical report](https://www.arxiv.org/abs/2510.15821).

#### Source Notebook: [chronos-2-quickstart.ipynb](https://github.com/amazon-science/chronos-forecasting/blob/main/notebooks/chronos-2-quickstart.ipynb)

## 0.0 Setup

In [ ]:
%pip install -U uv

In [ ]:
%uv pip install "sagemaker==2.253.1" "chronos-forecasting[extras]>=2.2" "matplotlib"

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

In [ ]:
region = boto3.Session().region_name

sess = sagemaker.Session(boto3.Session(region_name=region))

sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()

# Initialize SageMaker client
sm_client = boto3.client('sagemaker', region_name=region)

In [ ]:
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

## 1.0 Deploy `amazon/chronos-2` from SageMaker JumpStart

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from sagemaker.jumpstart.model import JumpStartModel
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models

In [ ]:
all_models = list_jumpstart_models(filter="task == forecasting")

# Filter for Chronos2 models
chronos2_models = [model for model in all_models if 'chronos' in model.lower()]

# Create dropdown widget
model_dropdown = widgets.Dropdown(
    options=chronos2_models,
    description='Chronos Model:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

# Display the dropdown
print(f"Found {len(chronos2_models)} Chronos models:")
display(model_dropdown)

# Access selected model
print(f"> Current model selection -> {model_dropdown.value}")

In [ ]:
print(f"Selected model: {model_dropdown.value}")

In [ ]:
model_id = model_dropdown.value
model_version = "*"  # Use latest version

# Create model instance
model = JumpStartModel(
    model_id=model_id,
    model_version=model_version,
    role=role,
    sagemaker_session=sess
)

In [ ]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.xlarge",  # Adjust instance type based on your needs
    endpoint_name=f"chronos2-endpoint-{sagemaker.utils.sagemaker_timestamp()}"
)

print(f"\nModel deployed successfully!")

## 2.0 Inference

Attach a predeployed SageMaker endpoint

In [ ]:
from sagemaker import serializers, deserializers

predictor = sagemaker.Predictor(
    endpoint_name="<jumpstart-endpoint>",
    sagemaker_session=sagemaker.Session(),
    serializer=serializers.JSONSerializer(),
    deserializer=deserializers.JSONDeserializer(),
)

#### Example 1: Sample Inference with `amazon/chronos-2` SageMaker Endpoint

In [ ]:
import json

In [ ]:
sample_input = {
    "inputs": [
        {
            "target": [0.0, 4.0, 5.0, 1.5, -3.0, -5.0, -3.0, 1.5, 5.0, 4.0],
            "past_covariates": {
                "feat_1": [0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 2.0, 0.0, 3.0, 1.0],
                "feat_2": ["A", "A", "A", "B", "B", "A", "B", "C", "A", "A"],
                "feat_3": [1.2, 0.5, 2.1, 1.8, 0.7, 1.5, 0.9, 2.3, 1.1, 2.2]
            },
            "future_covariates": {
                "feat_1": [0.5, 1.0, 1.0],
                "feat_2": ["C", "B", "C"]
            }
        }
    ],
    "parameters": {
        "prediction_length": 3,
        "quantile_levels": [0.1, 0.5, 0.9]
    }
}

In [ ]:
# Make prediction
response = predictor.predict(sample_input)
print(f"\nPrediction: {response}")

In [ ]:
## Clean up (optional - uncomment to delete endpoint after use)
# predictor.delete_endpoint()
# print("Endpoint deleted")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Example 2: Energy Price Forecasting (with covariates)

In [ ]:
# Visualization helper function
def plot_forecast(
    context_df: pd.DataFrame,
    pred_df: pd.DataFrame,
    test_df: pd.DataFrame,
    target_column: str,
    timeseries_id: str,
    id_column: str = "id",
    timestamp_column: str = "timestamp",
    history_length: int = 256,
    title_suffix: str = "",
):
    ts_context = context_df.query(f"{id_column} == @timeseries_id").set_index(timestamp_column)[target_column]
    ts_pred = pred_df.query(f"{id_column} == @timeseries_id and target_name == @target_column").set_index(
        timestamp_column
    )[["0.1", "predictions", "0.9"]]
    ts_ground_truth = test_df.query(f"{id_column} == @timeseries_id").set_index(timestamp_column)[target_column]

    last_date = ts_context.index.max()
    start_idx = max(0, len(ts_context) - history_length)
    plot_cutoff = ts_context.index[start_idx]
    ts_context = ts_context[ts_context.index >= plot_cutoff]
    ts_pred = ts_pred[ts_pred.index >= plot_cutoff]
    ts_ground_truth = ts_ground_truth[ts_ground_truth.index >= plot_cutoff]

    fig = plt.figure(figsize=(12, 3))
    ax = fig.gca()
    ts_context.plot(ax=ax, label=f"historical {target_column}", color="xkcd:azure")
    ts_ground_truth.plot(ax=ax, label=f"future {target_column} (ground truth)", color="xkcd:grass green")
    ts_pred["predictions"].plot(ax=ax, label="forecast", color="xkcd:violet")
    ax.fill_between(
        ts_pred.index,
        ts_pred["0.1"],
        ts_pred["0.9"],
        alpha=0.7,
        label="prediction interval",
        color="xkcd:light lavender",
    )
    ax.axvline(x=last_date, color="black", linestyle="--", alpha=0.5)
    ax.legend(loc="upper left")
    ax.set_title(f"{target_column} forecast for {timeseries_id} {title_suffix}")
    fig.show()

In [ ]:
target = "target"  # Column name containing the values to forecast (energy prices)
prediction_length = 24  # Number of hours to forecast ahead
id_column = "id"  # Column identifying different time series (countries/regions)
timestamp_column = "timestamp"  # Column containing datetime information
timeseries_id = "DE"  # Specific time series to visualize (Germany)

# Load historical energy prices and past values of covariates
energy_context_df = pd.read_parquet(
    "https://autogluon.s3.amazonaws.com/datasets/timeseries/electricity_price/train.parquet"
)
energy_context_df[timestamp_column] = pd.to_datetime(energy_context_df[timestamp_column])
display(energy_context_df.head())

# Load future values of covariates
energy_test_df = pd.read_parquet(
    "https://autogluon.s3.amazonaws.com/datasets/timeseries/electricity_price/test.parquet"
)
energy_test_df[timestamp_column] = pd.to_datetime(energy_test_df[timestamp_column])
energy_future_df = energy_test_df.drop(columns=target)
display(energy_future_df.head())

In [ ]:
energy_context_df_ = energy_context_df.copy()
energy_future_df_ = energy_future_df.copy()
energy_context_df_ = energy_context_df_.drop(columns=timestamp_column)
energy_future_df_ = energy_future_df_.drop(columns=timestamp_column)

target_ref = energy_context_df_[target].to_list()
x_ref = energy_context_df_.drop(columns=[target]).to_dict(orient="list")

x_future = energy_future_df_.to_dict(orient="list")

In [ ]:
energy_inputs_with_covars = {
    "inputs": [
        {
            "target": target_ref,
            "past_covariates": x_ref,
            "future_covariates": x_future
        }
    ],
    "parameters": {
        "prediction_length": prediction_length,
        "quantile_levels": [0.1, 0.5, 0.9]
    }
}

In [ ]:
# Make prediction
response = predictor.predict(energy_inputs_with_covars)
preds_df = pd.DataFrame(response["predictions"][0])

In [ ]:
energy_preds_df = pd.concat([energy_future_df, preds_df], axis=1).rename(columns={"mean": "predictions"})
energy_preds_df["target_name"] = "target"

In [ ]:
energy_context_df[timestamp_column] = pd.to_datetime(energy_context_df[timestamp_column])
energy_preds_df[timestamp_column] = pd.to_datetime(energy_preds_df[timestamp_column])

plot_forecast(
    energy_context_df,
    energy_preds_df,
    energy_test_df,
    target_column="target",
    timeseries_id=timeseries_id,
    title_suffix="(with covariates)",
)

#### Example 3: Energy Price Forecasting (without covariates)

In [ ]:
energy_inputs_without_covars = {
    "inputs": [
        {
            "target": target_ref,
            "past_covariates": x_ref,
            # "future_covariates": x_future
        }
    ],
    "parameters": {
        "prediction_length": prediction_length,
        "quantile_levels": [0.1, 0.5, 0.9]
    }
}

In [ ]:
# Make prediction
response = predictor.predict(energy_inputs_without_covars)
preds_df = pd.DataFrame(response["predictions"][0])

In [ ]:
energy_preds_df = pd.concat([energy_future_df, preds_df], axis=1).rename(columns={"mean": "predictions"})
energy_preds_df["target_name"] = target

In [ ]:
energy_context_df[timestamp_column] = pd.to_datetime(energy_context_df[timestamp_column])
energy_preds_df[timestamp_column] = pd.to_datetime(energy_preds_df[timestamp_column])

plot_forecast(
    energy_context_df,
    energy_preds_df,
    energy_test_df,
    target_column=target,
    timeseries_id=timeseries_id,
    title_suffix="(without covariates)",
)

## 4.0 Fine-Tune `amazon/chronos-2` using Amazon SageMaker AI's `ModelTrainer`

### 4.1 Load [amazon/chronos-2](https://huggingface.co/amazon/chronos-2) locally

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from chronos import BaseChronosPipeline, Chronos2Pipeline

In [ ]:
pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-2", 
    device_map="cpu" # or device_map="cuda"
)

### 4.2 Forecast with Sample Data

In [ ]:
target = "Sales"  # Column name containing sales values to forecast
prediction_length = 13  # Number of days to forecast ahead
id_column = "id"  # Column identifying different products/stores
timestamp_column = "timestamp"  # Column containing datetime information
timeseries_id = "1"  # Specific time series to visualize (product/store ID)

# Load historical sales and past values of covariates
sales_context_df = pd.read_parquet("https://autogluon.s3.amazonaws.com/datasets/timeseries/retail_sales/train.parquet")
sales_context_df[timestamp_column] = pd.to_datetime(sales_context_df[timestamp_column])
display(sales_context_df.head())

# Load future values of covariates
sales_test_df = pd.read_parquet("https://autogluon.s3.amazonaws.com/datasets/timeseries/retail_sales/test.parquet")
sales_test_df[timestamp_column] = pd.to_datetime(sales_test_df[timestamp_column])
sales_future_df = sales_test_df.drop(columns=target)
display(sales_future_df.head())

#### Predictions *with* time series co-variates

In [ ]:
# Generate predictions with covariates
sales_pred_df = pipeline.predict_df(
    sales_context_df,
    future_df=sales_future_df,
    prediction_length=prediction_length,
    quantile_levels=[0.1, 0.5, 0.9],
    id_column=id_column,
    timestamp_column=timestamp_column,
    target=target,
)
# display(sales_pred_df.head())

# Visualize forecast with covariates
plot_forecast(
    sales_context_df,
    sales_pred_df,
    sales_test_df,
    target_column=target,
    timeseries_id=timeseries_id,
    title_suffix="(with covariates)",
)

#### Predictions *without* time series co-variates

In [ ]:
# Compare: forecast without covariates
sales_pred_no_cov_df = pipeline.predict_df(
    sales_context_df[[id_column, timestamp_column, target]],
    future_df=None,
    prediction_length=prediction_length,
    quantile_levels=[0.1, 0.5, 0.9],
    id_column=id_column,
    timestamp_column=timestamp_column,
    target=target,
)

plot_forecast(
    sales_context_df,
    sales_pred_no_cov_df,
    sales_test_df,
    target_column=target,
    timeseries_id=timeseries_id,
    title_suffix="(without covariates)",
)

### 4.3 Data Prep and Upload to S3

In [ ]:
from datetime import datetime
from sagemaker.s3 import S3Uploader
from datasets import Dataset, load_dataset

In [ ]:
known_covariates = ["Open", "Promo", "SchoolHoliday", "StateHoliday"]
past_covariates = ["Customers"]

train_inputs = []
for item_id, group in sales_context_df.groupby("id"):
    train_inputs.append({
        "target": group[target].values,
        "past_covariates": {col: group[col].values for col in past_covariates + known_covariates},
        # Future values of covariates are not used during training.
        # However, we need to include their names to indicate that these columns will be available at prediction time
        "future_covariates": {col: None for col in known_covariates},
    })

In [ ]:
dataset_train = Dataset.from_list(train_inputs)
training_dataset_path = "./train.jsonl"
dataset_train.to_json(training_dataset_path, lines=True)

#### Upload Training dataset to S3

In [ ]:
data_s3_uri = f"s3://{sess.default_bucket()}/tool-calling/grpo/qwen3-06B/{datetime.now().strftime('%Y%m%d%H%M%S')}"

uploaded_s3_uri = S3Uploader.upload(
    local_path=training_dataset_path,
    desired_s3_uri=data_s3_uri
)
print(f"Uploaded {training_dataset_path} to > {uploaded_s3_uri}")

### 4.4 Fine-tune a model using Amazon SageMaker AI `ModelTrainer`

Chronos-2 supports fine-tuning on your own data. You may either fine-tune all weights of the model (_full fine-tuning_) or a [low rank adapter (LoRA)](https://huggingface.co/docs/peft/en/package_reference/lora), which significantly reduces the number of trainable parameters.
    
The `fit` method accepts:
- `inputs`: Time series for fine-tuning (same format as predict_quantiles)
- `finetune_mode`: `"full"` or `"lora"`
- `lora_config`: The [`LoraConfig`](https://huggingface.co/docs/peft/en/package_reference/lora#peft.LoraConfig), in case `finetune_mode="lora"`
- `prediction_length`: Forecast horizon for fine-tuning
- `validation_inputs`: Optional validation data (same format as inputs)
- `learning_rate`: Optimizer learning rate (default: 1e-6, we recommend a higher learning rate such as 1e-5 for LoRA)
- `num_steps`: Number of training steps (default: 1000)
- `batch_size`: Batch size for training (default: 256)

Returns a new pipeline with the fine-tuned model.

Please read the docstring for details about specific arguments.

In [ ]:
import time
from sagemaker.modules.configs import (
    CheckpointConfig,
    Compute,
    OutputDataConfig,
    SourceCode,
    StoppingCondition,
)
from sagemaker.modules.configs import InputData
from sagemaker.modules.train import ModelTrainer
from getpass import getpass
import yaml
from jinja2 import Template

In [ ]:
MODEL_ID = "amazon/chronos-2"

In [ ]:
MLFLOW_TRACKING_SERVER_ARN = "arn:aws:sagemaker:<region>:01234567890:mlflow-tracking-server/my-mlflow-test-server-1"

if MLFLOW_TRACKING_SERVER_ARN:
    reports_to = "mlflow"
else:
    reports_to = "tensorboard"

In [ ]:
job_name = MODEL_ID.replace('/', '--').replace('.', '-')

In [ ]:
if MLFLOW_TRACKING_SERVER_ARN:
    training_env = {
        # mlflow tracking metrics
        "MLFLOW_EXPERIMENT_NAME": job_name,
        "MLFLOW_TAGS": json.dumps(
            {
                "source.job": "sm-training-jobs", 
                "source.model.name": "amazon-chronos-2", 
                "source.framework": "pytorch"
            }
        ),
        "MLFLOW_TRACKING_URI": MLFLOW_TRACKING_SERVER_ARN,
        "MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING": "true"
    }

In [ ]:
%%writefile sagemaker_code/requirements.txt
chronos-forecasting
torch>=2.0.0
transformers>=4.40.0
accelerate>=0.30.0
datasets>=2.18.0
peft>=0.10.0
mlflow>=2.10.0
numpy>=1.24.0
pandas>=2.0.0
psutil
nvidia-ml-py
sagemaker-mlflow==0.1.0
mlflow

In [ ]:
args = [
    "recipes/amazon/chronos-2--full.yaml"
]

training_instance_type = "ml.g6.2xlarge"
training_instance_count = 1

In [ ]:
pytorch_image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=sess.boto_session.region_name,
    version="2.8.0",
    instance_type=training_instance_type,
    image_scope="training",
)
print(f"Using image: {pytorch_image_uri}")

In [ ]:
source_code = SourceCode(
    source_dir="./sagemaker_code",
    command=f"bash train.sh {' '.join(args)}",
)

compute_configs = Compute(
    instance_type=training_instance_type,
    instance_count=training_instance_count,
    keep_alive_period_in_seconds=1800,
    volume_size_in_gb=25
)

base_job_name = f"{job_name}-finetune"
output_path = f"s3://{sess.default_bucket()}/{base_job_name}"

model_trainer = ModelTrainer(
    training_image=pytorch_image_uri,
    source_code=source_code,
    base_job_name=base_job_name,
    compute=compute_configs,
    stopping_condition=StoppingCondition(max_runtime_in_seconds=36000),
    output_data_config=OutputDataConfig(
        s3_output_path=output_path,
        compression_type="NONE",
    ),
    checkpoint_config=CheckpointConfig(
        s3_uri=os.path.join(
            output_path,
            "time-series", 
            job_name,
            "checkpoints"
        ), 
        local_path="/opt/ml/checkpoints"
    ),
    role=role,
    environment=training_env
)

In [ ]:
model_trainer.train(
    input_data_config=[
        InputData(
            channel_name="training",
            data_source=uploaded_s3_uri,  
        )
    ], 
    wait=False
)

#### Query a completed training job for model weight

In [ ]:
from sagemaker.estimator import Estimator

pytorch_estimator = Estimator.attach(
    training_job_name="<my-completed-training-job-name>"
)

s3_model_data_uri = pytorch_estimator.model_data
print(f"Fine-tuned model location: {s3_model_data_uri}")

In [ ]:
!aws s3 ls "{s3_model_data_uri["S3DataSource"]["S3Uri"]}amazon/chronos-2/"

### 4.5 Model Evaluation post fine-tuning

In [ ]:
from sagemaker.s3 import S3Downloader

In [ ]:
model_s3_uri = os.path.join(s3_model_data_uri["S3DataSource"]["S3Uri"], "amazon", "chronos-2")
local_path = "amazon-chronos-2-finetuned"

S3Downloader.download(
    s3_uri=model_s3_uri,
    local_path=local_path
)
print(f"Download from {model_s3_uri} to > {local_path}")

In [ ]:
finetuned_pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained(
    f"{local_path}/", 
    device_map="cpu" # or device_map="cuda"
)

In [ ]:
# Use the fine-tuned model for predictions
finetuned_pred_df = finetuned_pipeline.predict_df(
    sales_context_df,
    future_df=sales_future_df,
    prediction_length=13,
    quantile_levels=[0.1, 0.5, 0.9],
    id_column="id",
    timestamp_column="timestamp",
    target="Sales",
)

plot_forecast(
    sales_context_df,
    finetuned_pred_df,
    sales_test_df,
    target_column="Sales",
    timeseries_id="1",
    title_suffix="(full fine-tuned)",
)

### 5.0 Deploy Fine-tuned model to SageMaker Endpoint for Real-time inference

In [ ]:
import boto3
import sagemaker
import json
import os
import tarfile
from datetime import datetime

In [ ]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
region = sess.boto_region_name

print(f"Role: {role}")
print(f"Bucket: {bucket}")
print(f"Region: {region}")

In [ ]:
# Update serving.properties
serving_properties = f"""engine=Python
option.model_id={model_s3_uri}
option.dtype=fp32
option.task=custom
option.rolling_batch=disable
"""

with open("sagemaker_code/djl_inference/serving.properties", "w") as f:
    f.write(serving_properties)

print("Updated serving.properties:")
print(serving_properties)

In [ ]:
# Create tarball of inference code
model_tar_path = "chronos2-inference.tar.gz"

with tarfile.open(model_tar_path, "w:gz") as tar:
    tar.add("sagemaker_code/djl_inference/model.py", arcname="model.py")
    tar.add("sagemaker_code/djl_inference/serving.properties", arcname="serving.properties")
    tar.add("sagemaker_code/djl_inference/requirements.txt", arcname="requirements.txt")

print(f"Created {model_tar_path}")

# Upload to S3
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
s3_code_uri = f"s3://{bucket}/chronos2-finetuned/inference-code/{timestamp}/model.tar.gz"

s3_client = boto3.client("s3")
s3_client.upload_file(model_tar_path, bucket, s3_code_uri.replace(f"s3://{bucket}/", ""))

print(f"Uploaded to: {s3_code_uri}")

In [ ]:
from sagemaker.djl_inference import DJLModel

# DJL DeepSpeed container for GPU inference
djl_image_uri = sagemaker.image_uris.retrieve(
    framework="djl-deepspeed",
    region=region,
    version="0.27.0"
)

print(f"Using image: {djl_image_uri}")

In [ ]:
model_name = f"chronos2-finetuned-{timestamp}"

model = DJLModel(
    model_data=s3_code_uri,
    role=role,
    image_uri=djl_image_uri,
    sagemaker_session=sess,
    name=model_name,
)

print(f"Model created: {model_name}")

In [ ]:
endpoint_name = f"chronos2-finetuned-endpoint-{timestamp}"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.xlarge",  # GPU instance
    endpoint_name=endpoint_name,
    sagemaker_session=sess
)

print(f"Endpoint deployed: {endpoint_name}")

In [ ]:
import pandas as pd
import boto3

# Config
target = "Sales"
prediction_length = 13
id_column = "id"
timestamp_column = "timestamp"
timeseries_id = "1"

# Load data
sales_context_df = pd.read_parquet("https://autogluon.s3.amazonaws.com/datasets/timeseries/retail_sales/train.parquet")
sales_context_df[timestamp_column] = pd.to_datetime(sales_context_df[timestamp_column])

sales_test_df = pd.read_parquet("https://autogluon.s3.amazonaws.com/datasets/timeseries/retail_sales/test.parquet")
sales_test_df[timestamp_column] = pd.to_datetime(sales_test_df[timestamp_column])
sales_future_df = sales_test_df.drop(columns=target)

# Save locally
sales_context_df.to_parquet("sales_context.parquet")
sales_future_df.to_parquet("sales_future.parquet")

# Upload to S3
s3_client = boto3.client("s3")
s3_prefix = "chronos2-test/retail-sales"

s3_client.upload_file("sales_context.parquet", bucket, f"{s3_prefix}/context.parquet")
s3_client.upload_file("sales_future.parquet", bucket, f"{s3_prefix}/future.parquet")

context_s3_uri = f"s3://{bucket}/{s3_prefix}/context.parquet"
future_s3_uri = f"s3://{bucket}/{s3_prefix}/future.parquet"

print(f"Context uploaded: {context_s3_uri}")
print(f"Future uploaded: {future_s3_uri}")

In [ ]:
test_input = {
    "context_data": context_s3_uri,
    "future_data": future_s3_uri,
    "parameters": {
        "prediction_length": prediction_length,
        "quantile_levels": [0.1, 0.5, 0.9],
        "id_column": id_column,
        "timestamp_column": timestamp_column,
        "target": target
    }
}

response = predictor.predict(test_input)

print(f"Status: {response.get('status')}")
print(f"Predictions: {len(response.get('predictions', []))}")
print(f"Num series: {response.get('num_series')}")

In [ ]:
pred_df = pd.DataFrame(response["predictions"])
pred_df[timestamp_column] = pd.to_datetime(pred_df[timestamp_column])

plot_forecast(
    sales_context_df,
    pred_df,
    sales_test_df,
    target_column=target,
    timeseries_id=timeseries_id,
    id_column=id_column,
    timestamp_column=timestamp_column,
    title_suffix="(fine-tuned endpoint - with covariates)",
)